In [1]:
import cv2 as cv
import numpy as np
import IPython.display as ipd

In [4]:
# 1. read in the class names and assign random colors to each
with open('input/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().strip().split('\n')

colors = np.random.uniform(0, 255, size=(len(class_names), 3))

In [9]:
# 2. load the model
model = cv.dnn.readNet(
    model='input/frozen_inference_graph.pb',
    config='input/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',
    framework='TensorFlow'
)

In [11]:
# 3. read in the image and prepare it for model input
image = cv.imread('input/image_2.jpg')
height, width, _ = image.shape

blob = cv.dnn.blobFromImage(
    image=image,
    size=(300, 300),
    mean=(104, 117, 123),
    swapRB=True,  # openCV reads BGR color, so this swaps to RGB to match the model
)
model.setInput(blob)
output = model.forward()

**Output structure:**
```
[[0, class_label, confidence_score, bbox_x, bbox_y, bbox_width, bbox_height], ...]
```

- I don't know what the initial `0` does of if that ever changes
- `class_label` is the index of the class label (an integer between 1 and 80)
- `confidence_score` is not a direct probability score, but is a measure of the model's confidence in its detection
- `bbox_x`, `bbox_y`, `bbox_width`, `bbox_height` describe the bounding box for the object

In [ ]:
# Note, the shape is (1, 1, 100, 7), so the first two axes do nothing
for detection in output[0, 0, :, :]:
    conf_score = detection[2] # get the confidence score
    if conf_score > 0.4:
        class_id = int(detection[1]) - 1
        class_name = class_names[class_id]
        color = colors[class_id]
        bbox_x = int(detection[3] * width)
        bbox_y = int(detection[4] * height)
        bbox_width = int(detection[5] * width)
        bbox_height = int(detection[6] * height)
        cv.rectangle(image, (bbox_x, bbox_y), (bbox_width, bbox_height), color, thickness=2)
        cv.putText(image, class_name, (bbox_x, bbox_y - 5), cv.FONT_HERSHEY_SIMPLEX, 1, color, 2)

cv.imshow('image', image)
cv.imwrite('image_result.jpg', image)
cv.waitKey()
cv.destroyAllWindows()